In [81]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

#set the max columns to none
pd.set_option('display.max_columns', None)

### I will create the interaction matrix from history data and save it in order to quick access

In [118]:
size = 'demo'
type_ = 'train'
#Import train Data
history_train_path = f'./files/parquet/ebnerd_{size}/{type_}/history.parquet'

history_df = pd.read_parquet(history_train_path)
print(history_df.shape)

behaviors_path = f'./files/parquet/ebnerd_{size}/{type_}/behaviors.parquet'
behaviors_df = pd.read_parquet(behaviors_path)
print(f'Raw {size} validation behaviors df shape:          ',behaviors_df.shape)


(1590, 5)
Raw demo validation behaviors df shape:           (24724, 17)


In [119]:
print(history_df.explode('impression_time_fixed').impression_time_fixed.min())
print(history_df.explode('impression_time_fixed').impression_time_fixed.max())
history_df.head(2)

2023-04-27 07:00:05
2023-05-18 06:59:51


,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,13538,"[2023-04-27T10:17:43.000000, 2023-04-27T10:18:...","[100.0, 35.0, 100.0, 24.0, 100.0, 23.0, 100.0,...","[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11..."
1,58608,"[2023-04-27T18:48:09.000000, 2023-04-27T18:48:...","[37.0, 61.0, 100.0, 100.0, 55.0, 100.0, 100.0,...","[9739362, 9739179, 9738567, 9739344, 9739202, ...","[2.0, 24.0, 72.0, 65.0, 11.0, 4.0, 101.0, 0.0,..."


In [120]:
print(behaviors_df.impression_time.min())
print(behaviors_df.impression_time.max())
behaviors_df.head(2)

2023-05-18 07:00:03
2023-05-25 06:59:52


,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,48401,NaN,2023-05-21 21:06:50,21.0,NaN,2,"[9774516, 9771051, 9770028, 9775402, 9774461, ...",[9759966],22779,False,NaN,NaN,NaN,False,21,16.0,27.0
1,152513,9778745.0,2023-05-24 07:31:26,30.0,100.0,1,"[9778669, 9778736, 9778623, 9089120, 9778661, ...",[9778661],150224,False,NaN,NaN,NaN,False,298,2.0,48.0


In [121]:
# Preprocess behaviour
behaviors_df = behaviors_df[['article_ids_clicked','user_id','read_time']]
behaviors_df = behaviors_df.explode('article_ids_clicked')


In [122]:
behaviors_df.groupby('user_id')['article_ids_clicked'].apply(list).reset_index()


,user_id,article_ids_clicked
0,11313,"[9776897, 9779860, 9778627, 9776855, 9776715, ..."
1,13538,"[9754160, 9771187, 9773768, 9772343, 9775171, ..."
2,15430,"[9761586, 9778375, 9770028]"
3,19181,"[9776385, 9777636, 9774764, 9777475, 9773018, ..."
4,19568,"[9779713, 9772284]"
...,...,...
1585,2581162,"[9777693, 9775846, 9777565, 9775785, 9775722, ..."
1586,2583035,"[9775873, 9772104, 9773316, 9774532, 9774923, ..."
1587,2583477,"[9779007, 9778939, 9744403, 9738729, 9772629, ..."
1588,2585449,"[9778769, 9778952, 9775733, 9750696, 9776337, ..."


In [123]:
behaviors_df.groupby('user_id')['read_time'].apply(list).reset_index()


,user_id,read_time
0,11313,"[11.0, 21.0, 51.0, 4.0, 12.0, 9.0, 108.0, 13.0]"
1,13538,"[11.0, 18.0, 12.0, 25.0, 8.0, 26.0, 28.0, 6.0,..."
2,15430,"[21.0, 12.0, 146.0]"
3,19181,"[22.0, 17.0, 53.0, 22.0, 12.0, 30.0, 12.0, 11...."
4,19568,"[12.0, 9.0]"
...,...,...
1585,2581162,"[36.0, 10.0, 26.0, 37.0, 4.0, 111.0, 24.0, 7.0..."
1586,2583035,"[21.0, 26.0, 20.0, 13.0, 9.0, 28.0, 78.0, 78.0..."
1587,2583477,"[133.0, 20.0, 33.0, 27.0, 10.0, 21.0, 23.0, 16..."
1588,2585449,"[28.0, 670.0, 7.0, 89.0, 9.0, 6.0, 41.0, 10.0,..."


In [124]:
behaviors_df= pd.merge(behaviors_df.groupby('user_id')['article_ids_clicked'].apply(list).reset_index(),behaviors_df.groupby('user_id')['read_time'].apply(list).reset_index(),on='user_id')

In [125]:
behaviors_df.head(2)

,user_id,article_ids_clicked,read_time
0,11313,"[9776897, 9779860, 9778627, 9776855, 9776715, ...","[11.0, 21.0, 51.0, 4.0, 12.0, 9.0, 108.0, 13.0]"
1,13538,"[9754160, 9771187, 9773768, 9772343, 9775171, ...","[11.0, 18.0, 12.0, 25.0, 8.0, 26.0, 28.0, 6.0,..."


In [126]:
try:
    history_df.drop('impression_time_fixed', axis=1, inplace=True) # i will drop this column initialy even though i can proccess it and use it
except:
    pass

In [127]:
history_df['scroll_percentage_fixed_len'] = history_df['scroll_percentage_fixed'].apply(lambda lst: len(lst))
history_df['article_id_fixed_len'] = history_df['article_id_fixed'].apply(lambda lst: len(lst))
history_df['read_time_fixed_len'] = history_df['read_time_fixed'].apply(lambda lst: len(lst))

In [128]:
history_df.head(2)

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed,scroll_percentage_fixed_len,article_id_fixed_len,read_time_fixed_len
0,13538,"[100.0, 35.0, 100.0, 24.0, 100.0, 23.0, 100.0,...","[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11...",582,582,582
1,58608,"[37.0, 61.0, 100.0, 100.0, 55.0, 100.0, 100.0,...","[9739362, 9739179, 9738567, 9739344, 9739202, ...","[2.0, 24.0, 72.0, 65.0, 11.0, 4.0, 101.0, 0.0,...",151,151,151


### The lenths of the lists in each column seem equal

In [129]:
history_df['scroll_percentage_fixed_has_0_in'] = history_df['scroll_percentage_fixed'].apply(lambda lst: 0 in lst)
history_df['read_time_fixed_has_0_in'] = history_df['read_time_fixed'].apply(lambda lst: 0 in lst)
history_df['read_time_fixed_has_nan'] = history_df['read_time_fixed'].apply(lambda lst: pd.isna(lst.any()))

In [130]:
history_df.head()

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed,scroll_percentage_fixed_len,article_id_fixed_len,read_time_fixed_len,scroll_percentage_fixed_has_0_in,read_time_fixed_has_0_in,read_time_fixed_has_nan
0,13538,"[100.0, 35.0, 100.0, 24.0, 100.0, 23.0, 100.0,...","[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11...",582,582,582,True,True,False
1,58608,"[37.0, 61.0, 100.0, 100.0, 55.0, 100.0, 100.0,...","[9739362, 9739179, 9738567, 9739344, 9739202, ...","[2.0, 24.0, 72.0, 65.0, 11.0, 4.0, 101.0, 0.0,...",151,151,151,False,True,False
2,95507,"[60.0, 100.0, 100.0, 21.0, 29.0, 67.0, 49.0, 5...","[9739035, 9738646, 9634967, 9738902, 9735495, ...","[18.0, 29.0, 51.0, 12.0, 10.0, 10.0, 13.0, 24....",370,370,370,False,True,False
3,106588,"[24.0, 57.0, 100.0, nan, nan, 100.0, 100.0, 73...","[9738292, 9738216, 9737266, 9737556, 9737657, ...","[9.0, 15.0, 42.0, 9.0, 3.0, 58.0, 26.0, 214.0,...",149,149,149,False,True,False
4,617963,"[100.0, 100.0, nan, 46.0, 23.0, 19.0, 61.0, 70...","[9739035, 9739088, 9738902, 9738968, 9738760, ...","[45.0, 29.0, 116.0, 26.0, 34.0, 42.0, 58.0, 59...",277,277,277,False,True,False


In [131]:
history_df['read_time_fixed_has_nan'].unique()

array([False])

### There both nulls and zeros in the scroll_percentage_fixed column but not in read_time_fixed, i will use the read_time_fixed for interraction matrix or i will have to fix the nans to scroll_percentage_fixed

In [132]:
# # Combine interactions into a single metric (Potential use of scroll_percentage_fixed column)
# def combine_interactions(interactions, interactions2, w1=0.5, w2=0.5):
#     return [w1*i + w2*i2  for i, i2 in zip(interactions, interactions2)]

# history_dfraction_combined'] = history_df(lambda row:   combine_interactions(row['scroll_percentage_fixed'], row['read_time_fixed']) , axis=1)

# history_df ['scroll_percentage_fixed', 'read_time_fixed'], axis=1, inplace=True)

In [133]:
history_df.drop(['scroll_percentage_fixed_len', 'article_id_fixed_len', 'read_time_fixed_len', 'scroll_percentage_fixed_has_0_in', 'read_time_fixed_has_nan', 'scroll_percentage_fixed','read_time_fixed_has_0_in'], axis=1, inplace=True)

In [134]:
history_df.head(2)

,user_id,article_id_fixed,read_time_fixed
0,13538,"[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11..."
1,58608,"[9739362, 9739179, 9738567, 9739344, 9739202, ...","[2.0, 24.0, 72.0, 65.0, 11.0, 4.0, 101.0, 0.0,..."


In [135]:
history_df.shape

(1590, 3)

In [136]:
behaviors_df.shape

(1590, 3)

In [137]:
## Merge behaviours and history-left join
final_df = pd.merge(behaviors_df,history_df,on='user_id',how='left')
final_df.shape

(1590, 5)

In [138]:
final_df.head()

,user_id,article_ids_clicked,read_time,article_id_fixed,read_time_fixed
0,11313,"[9776897, 9779860, 9778627, 9776855, 9776715, ...","[11.0, 21.0, 51.0, 4.0, 12.0, 9.0, 108.0, 13.0]","[9737535, 9738334, 9738292, 9733713, 9740161, ...","[1038.0, 4.0, 32.0, 7.0, 4.0, 34.0, 640.0, 9.0..."
1,13538,"[9754160, 9771187, 9773768, 9772343, 9775171, ...","[11.0, 18.0, 12.0, 25.0, 8.0, 26.0, 28.0, 6.0,...","[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11..."
2,15430,"[9761586, 9778375, 9770028]","[21.0, 12.0, 146.0]","[9745909, 9746802, 9748576, 9748470, 9748576, ...","[101.0, 0.0, 32.0, 70.0, 12.0, 136.0, 75.0, 0...."
3,19181,"[9776385, 9777636, 9774764, 9777475, 9773018, ...","[22.0, 17.0, 53.0, 22.0, 12.0, 30.0, 12.0, 11....","[9737521, 9738292, 9738216, 9737266, 9738182, ...","[2.0, 6.0, 1.0, 18.0, 24.0, 22.0, 4.0, 15.0, 1..."
4,19568,"[9779713, 9772284]","[12.0, 9.0]","[9740249, 9740237, 9739967, 9743702, 9743714, ...","[4.0, 2.0, 64.0, 6.0, 29.0, 0.0, 116.0, 8.0, 2..."


In [139]:
def concatenate_lists(df, col1, col2, new_col_name):
    df[new_col_name] = df.apply(lambda row: row[col1] + row[col2] if isinstance(row[col1], list) and isinstance(row[col2], list) else row[col1] if isinstance(row[col1], list) else row[col2] if isinstance(row[col2], list) else [], axis=1)
    return df

In [140]:
final_df = concatenate_lists(final_df,"article_ids_clicked","article_id_fixed","merge_article_ids")
final_df = concatenate_lists(final_df,"read_time","read_time_fixed","merge_read_time")

In [141]:
final_df.shape

(1590, 7)

In [142]:
final_df = final_df[['merge_article_ids','merge_read_time','user_id']]

In [143]:
final_df.head(2)

,merge_article_ids,merge_read_time,user_id
0,"[9776897, 9779860, 9778627, 9776855, 9776715, ...","[11.0, 21.0, 51.0, 4.0, 12.0, 9.0, 108.0, 13.0]",11313
1,"[9754160, 9771187, 9773768, 9772343, 9775171, ...","[11.0, 18.0, 12.0, 25.0, 8.0, 26.0, 28.0, 6.0,...",13538


In [144]:
# final_df.set_index('user_id',inplace=True)

In [145]:
# final_df.reset_index(inplace=True)

In [146]:
final_df = final_df.explode('merge_article_ids')

In [147]:
final_df.shape

(24888, 3)

In [148]:
final_df.head()

,merge_article_ids,merge_read_time,user_id
0,9776897,"[11.0, 21.0, 51.0, 4.0, 12.0, 9.0, 108.0, 13.0]",11313
0,9779860,"[11.0, 21.0, 51.0, 4.0, 12.0, 9.0, 108.0, 13.0]",11313
0,9778627,"[11.0, 21.0, 51.0, 4.0, 12.0, 9.0, 108.0, 13.0]",11313
0,9776855,"[11.0, 21.0, 51.0, 4.0, 12.0, 9.0, 108.0, 13.0]",11313
0,9776715,"[11.0, 21.0, 51.0, 4.0, 12.0, 9.0, 108.0, 13.0]",11313


In [149]:
# Creation of interaction matrix

interaction_matrix = pd.DataFrame(columns=final_df.columns)
counter = 0
for user_id in tqdm(np.unique(final_df.index.values).tolist()):
    
    temp_df = final_df[final_df.index == user_id].copy()
    temp_df = temp_df.explode('merge_article_ids')
    temp_df.set_index('user_id',inplace=True)
    temp_lst = []

    # Fix the lists that exist in each cell and keep the values that i care
    temp_counter = 0
    for lst in list(temp_df['merge_read_time']):
        temp_lst.append(lst[temp_counter])
        temp_counter += 1
    
    temp_df['merge_read_time_fixed'] = temp_lst

    # Drop first unneccesary column
    temp_df.drop('merge_read_time', axis=1, inplace= True)
    temp_pivoted_df = temp_df.pivot_table(index = temp_df.index, columns='merge_article_ids', values='merge_read_time_fixed', aggfunc='sum').copy()

    if counter == 0:
        interaction_matrix = temp_pivoted_df.copy()
        #print(interaction_matrix)
    else:
        interaction_matrix = pd.concat([interaction_matrix, temp_pivoted_df], axis=0, ignore_index=False)
    
    counter += 1

C:\Users\giwrg\AppData\Local\Temp\ipykernel_30928\1750402660.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for user_id in tqdm(np.unique(final_df.index.values).tolist()):


  0%|          | 0/1590 [00:00<?, ?it/s]

In [150]:
interaction_matrix.shape

(1590, 1114)

In [151]:
interaction_matrix.head()

merge_article_ids,9775489,9775567,9776715,9776855,9776897,9778627,9778701,9779860,9754160,9768308,9771187,9771351,9772343,9772453,9772869,9773768,9774404,9774430,9775142,9775171,9761586,9770028,9778375,9466422,9762122,9771126,9771367,9771572,9771626,9772029,9772485,9773018,9773673,9773852,9774574,9774764,9776385,9777475,9777636,9777811,9778110,9778328,9779007,9780020,9772284,9779713,9735909,9770882,9770082,9775785,9775964,9776303,9772434,9772866,9773493,9773612,9775983,9776442,9772545,9759966,9769800,9771168,9773364,9773887,9774595,9776322,9776489,9777164,9777992,9778277,9778351,9778413,9779511,9779577,9779737,9767470,9768043,9770729,9770886,9770997,9771042,9771254,9771554,9772050,9772302,9772380,9772389,9772668,9772835,9773316,9773447,9773644,9773846,9773987,9774096,9774229,9774352,9774506,9774532,9774598,9774618,9774937,9775202,9776046,9776176,9776278,9777292,9778074,9778381,9779288,9779747,9759955,9765941,9769370,9769497,9769996,9770568,9771166,9771479,9771627,9771842,9771938,9771948,9772088,9772115,9772219,9772279,9772557,9772635,9772957,9773351,9773617,9773727,9775056,9775388,9775461,9775793,9775800,9775881,9775905,9775998,9776071,9776148,9776449,9776688,9776882,9777321,9777393,9777941,9778326,9778661,9778669,9778722,9778731,9778804,9778843,9778875,9778971,9779045,9779674,9779705,9729924,9772095,9772168,9775855,9775939,9776041,9776087,9776234,9776259,9776386,9776394,9746810,9769433,9769504,9769581,9771121,9771235,9771473,9771775,9771855,9772099,9772300,9772502,9772710,9772805,9772873,9773296,9774074,9774321,9774542,9774554,9774826,9774840,9775183,9775803,9775804,9775846,9776023,9776049,9776553,9776691,9777464,9778168,9778310,9778902,9773470,9775809,9776420,9777648,9778198,9770045,9770292,9771124,9771127,9771579,9771686,9771764,9771846,9772830,9773070,9773376,9767989,9768866,9771916,9771995,9774557,9775500,9775611,9775621,9776566,9778302,9780019,9758424,9772442,9777520,9767490,9772032,9775716,9775722,9769624,9771113,9772227,9772228,9772407,9772433,9774516,9779797,9759544,9769769,9771325,9771333,9771576,9771779,9771796,9771999,9772000,9772026,9772706,9775361,9775493,9775703,9776184,9778710,9778917,9778939,9778942,9779423,9744403,9760046,9774392,9774708,9775347,9775648,9772632,9775323,9766757,9770451,9772291,9775485,9775990,9776246,9776808,9776864,9778007,9778021,9778386,9778448,9728166,9766140,9772263,9778745,9769605,9771859,9772344,9773543,9773857,9774013,9774382,9774652,9775419,9776040,9776337,9776474,9777768,9777822,9779629,9772629,9777299,9775894,9776223,9778623,9776583,9778257,8315213,9531745,9535187,9747757,9755800,9761859,9769917,9770029,9770989,9771568,9771896,9772104,9772659,9772711,9773210,9773566,9773726,9774252,9774527,9775402,9775776,9775915,9775985,9776070,9776292,9777026,9777505,9779417,9779520,9779867,9779956,9775903,9776985,9769762,9779263,9768177,9772082,9772045,9772193,9773292,9773295,9773356,9773947,9774403,9777565,9767665,9770283,9771128,9771996,9775673,9775908,9776191,9776918,9777804,9779204,9779366,9779377,9482380,9755712,9761926,9769341,9769531,9770288,9771065,9771125,9771242,9772275,9772517,9772575,9772772,9773985,9774032,9774363,9775277,9775331,9775568,9775582,9775596,9775717,9776341,9776544,9777358,9777385,9777492,9778192,9778813,9779427,9779748,9779777,9780039,9770538,9771869,9773015,9769432,9771350,9459174,9737094,9742173,9763188,9770369,9772241,9772660,9772727,9772750,9772768,9772882,9772923,9772925,9772963,9773045,9773084,9775003,9753689,9773282,9774020,9774079,9772297,9772443,9772355,9772601,9775562,9776917,9779184,9779285,9772543,9775909,9778780,9771248,9771438,9774789,9770450,9771253,9771919,9773341,9773877,9778236,9775701,9775927,9776870,9694850,9768850,9769712,9775873,9777319,9769306,9772047,9773464,9779498,9391394,9508202,9566489,9738729,9766136,9767342,9768469,9769424,9770541,9772178,9772448,9773228,9773275,9773412,9773434,9773488,9773962,9774559,9774923,9775042,9776332,9776369,9776508,9776560,9776929,9777233,9777307,9777582,9778666,9778682,9778769,9778788,9779186,9486080,9774383,977940

In [152]:
fillna_value = '0'
interaction_matrix.fillna(0, inplace=True)

In [153]:
# Save the interaction matrix to pickle
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{type_}_{size}_{fillna_value}.pkl'

interaction_matrix.to_pickle(interaction_matrix_file_path)

In [45]:
# Read interaction matrix pickle file
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{size}_{fillna_value}.pkl'
interaction_matrix = pd.read_pickle(interaction_matrix_file_path)

FileNotFoundError: [Errno 2] No such file or directory: './files/pickle/interaction_matrix_demo_0.pkl'

## Other

In [ ]:
# # Combine interactions into a single metric
# def combine_interactions(interactions, interactions2, w1=1, w2=1):
#     return [w1*i + w2*i2  for i, i2 in zip(interactions, interactions2)]

# # Flatten the interaction data with combined interactions
# flattened_data = []

# for _, row in history_df.iterrows():
#     user_id = row['user_id']
#     article_ids = row['article_id_fixed']
#     combined_interactions = combine_interactions(row['scroll_percentage_fixed'], row['read_time_fixed'])
    
#     for article_id, interaction_combined in zip(article_ids, combined_interactions):
#         flattened_data.append({
#             'user_id': user_id, 
#             'article_id_fixed': article_id, 
#             'interaction_combined': interaction_combined
#         })

# # Convert to DataFrame
# interaction_df = pd.DataFrame(flattened_data)